In [24]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 3: Toboggan Trajectory ---</h2><p>With the toboggan login problems resolved, you set off toward the airport. While travel by toboggan might be easy, it's certainly not safe: there's <span title="It looks like the toboggan steering system even runs on Intcode! Good thing you don't have to modify it.">very minimal steering</span> and the area is covered in trees. You'll need to see which angles will take you near the fewest trees.</p>
<p>Due to the local geology, trees in this area only grow on exact integer coordinates in a grid. You make a map (your puzzle input) of the open squares (<code>.</code>) and trees (<code>#</code>) you can see. For example:</p>
<pre><code>..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
</code></pre>
<p>These aren't the only trees, though; due to something you read about once involving arboreal genetics and biome stability, the same pattern repeats to the right many times:</p>
<pre><code><em>..##.......</em>..##.........##.........##.........##.........##.......  ---&gt;
<em>#...#...#..</em>#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
<em>.#....#..#.</em>.#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
<em>..#.#...#.#</em>..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
<em>.#...##..#.</em>.#...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
<em>..#.##.....</em>..#.##.......#.##.......#.##.......#.##.......#.##.....  ---&gt;
<em>.#.#.#....#</em>.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
<em>.#........#</em>.#........#.#........#.#........#.#........#.#........#
<em>#.##...#...</em>#.##...#...#.##...#...#.##...#...#.##...#...#.##...#...
<em>#...##....#</em>#...##....##...##....##...##....##...##....##...##....#
<em>.#..#...#.#</em>.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#.#..#...#.#  ---&gt;
</code></pre>
<p>You start on the open square (<code>.</code>) in the top-left corner and need to reach the bottom (below the bottom-most row on your map).</p>
<p>The toboggan can only follow a few specific slopes (you opted for a cheaper model that prefers rational numbers); start by <em>counting all the trees</em> you would encounter for the slope <em>right 3, down 1</em>:</p>
<p>From your starting position at the top-left, check the position that is right 3 and down 1. Then, check the position that is right 3 and down 1 from there, and so on until you go past the bottom of the map.</p>
<p>The locations you'd check in the above example are marked here with <code><em>O</em></code> where there was an open square and <code><em>X</em></code> where there was a tree:</p>
<pre><code>..##.........##.........##.........##.........##.........##.......  ---&gt;
#..<em>O</em>#...#..#...#...#..#...#...#..#...#...#..#...#...#..#...#...#..
.#....<em>X</em>..#..#....#..#..#....#..#..#....#..#..#....#..#..#....#..#.
..#.#...#<em>O</em>#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#..#.#...#.#
.#...##..#..<em>X</em>...##..#..#...##..#..#...##..#..#...##..#..#...##..#.
..#.##.......#.<em>X</em>#.......#.##.......#.##.......#.##.......#.##.....  ---&gt;
.#.#.#....#.#.#.#.<em>O</em>..#.#.#.#....#.#.#.#....#.#.#.#....#.#.#.#....#
.#........#.#........<em>X</em>.#........#.#........#.#........#.#........#
#.##...#...#.##...#...#.<em>X</em>#...#...#.##...#...#.##...#...#.##...#...
#...##....##...##....##...#<em>X</em>....##...##....##...##....##...##....#
.#..#...#.#.#..#...#.#.#..#...<em>X</em>.#.#..#...#.#.#..#...#.#.#..#...#.#  ---&gt;
</code></pre>
<p>In this example, traversing the map using this slope would cause you to encounter <code><em>7</em></code> trees.</p>
<p>Starting at the top-left corner of your map and following a slope of right 3 and down 1, <em>how many trees would you encounter?</em></p>
</article>


In [25]:
from math import prod


example = """
    ..##.......
    #...#...#..
    .#....#..#.
    ..#.#...#.#
    .#...##..#.
    ..#.##.....
    .#.#.#....#
    .#........#
    #.##...#...
    #...##....#
    .#..#...#.#
"""


class Toboggan:
    def __init__(self, s: str) -> None:
        self.geology = [l.strip() for l in s.strip().splitlines()]
        self.rows, self.cols = len(self.geology), len(self.geology[0])

    def count_tree_on_path_down(self, d_row: int = 1, d_col: int = 3) -> int:
        row, col = 0, 0
        trees = 0
        while row < self.rows:
            if self.geology[row][col] == "#":
                trees += 1
            row += d_row
            col = (col + d_col) % self.cols

        return trees

    def count_tree_on_path_down_on_slopes(
        self, slopes=((1, 1), (1, 3), (1, 5), (1, 7), (2, 1))
    ) -> int:
        return prod(self.count_tree_on_path_down(d_r, d_c) for d_r, d_c in slopes)

    def __repr__(self) -> str:
        return "\n".join(self.geology)


assert Toboggan(example).count_tree_on_path_down() == 7

In [26]:
with open("../input/day3.txt") as f:
    puzzle = f.read()

print(f"Part I: {Toboggan(puzzle).count_tree_on_path_down()}")

Part I: 268


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>268</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Time to check the rest of the slopes - you need to minimize the probability of a sudden arboreal stop, after all.</p>
<p>Determine the number of trees you would encounter if, for each of the following slopes, you start at the top-left corner and traverse the map all the way to the bottom:</p>
<ul>
<li>Right 1, down 1.</li>
<li>Right 3, down 1. (This is the slope you already checked.)</li>
<li>Right 5, down 1.</li>
<li>Right 7, down 1.</li>
<li>Right 1, down 2.</li>
</ul>
<p>In the above example, these slopes would find <code>2</code>, <code>7</code>, <code>3</code>, <code>4</code>, and <code>2</code> tree(s) respectively; multiplied together, these produce the answer <code><em>336</em></code>.</p>
<p><em>What do you get if you multiply together the number of trees encountered on each of the listed slopes?</em></p>
</article>

</main>


In [27]:
assert Toboggan(example).count_tree_on_path_down_on_slopes() == 336

In [28]:
print(f"Part II: { Toboggan(puzzle).count_tree_on_path_down_on_slopes()}")

Part II: 3093068400


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3093068400</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
